In [1]:
import pandas as pd
import tqdm
import requests
from datasets import Dataset
import os
import getpass
from clearml import Task

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

from ragas import evaluate

/Users/m.averchenko/PycharmProjects/SpicyRAG/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
eval_data_path = "../../data/eval_dataset.xlsx"
eval_data = pd.read_excel(eval_data_path)
eval_data

,Unnamed: 0,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,411,Зачем проводят конкурс среди оленеводов в ХМАО?,['АВТОНОМНОГО ОКРУГА - ЮГРЫ НА КУБОК ГУБЕРНАТО...,Конкурс профессионального мастерства среди оле...,conditional,[{'file_path': '/home/cias/projects/other/raga...,True
1,454,"Как можно сэкономить энергию в ХМАО-Югре, моде...","['- 325,2 г у.т./кВт-ч, на Нижневартовской ГРЭ...",Для экономии энергии в Ханты-Мансийском автоно...,conditional,[{'file_path': '/home/cias/projects/other/raga...,True
2,453,Как переводится федеральный чин в округе?,"['статьи, - на одну ступень выше классного чин...",При поступлении на гражданскую службу автономн...,conditional,[{'file_path': '/home/cias/projects/other/raga...,True
3,439,"Кто может претендовать на премию, если работае...","['Пункт 1 изложить в следующей редакции: ""1. П...",На премию могут претендовать педагогические ра...,conditional,[{'file_path': '/home/cias/projects/other/raga...,True
4,429,Как округ решает противоречия между местными и...,['и) организует контроль за выполнением недроп...,Округ вправе предлагать органам местного самоу...,conditional,[{'file_path': '/home/cias/projects/other/raga...,True
...,...,...,...,...,...,...,...
115,54,Где предполагается установка систем видеонаблю...,['|1. |Оснащение |Департамент |2009 - |5000 |-...,Системы видеонаблюдения с установкой мониторов...,simple,[{'file_path': '/home/cias/projects/other/raga...,True
116,78,Когда и где пройдет выездная лекция-концерт «С...,['смете расходов | | |образования автономного ...,Выездная лекция-концерт «С чего начинается Род...,simple,[{'file_path': '/home/cias/projects/other/raga...,True
117,45,Какие меры социальной защиты предусмотрены для...,['|Задача: организация правовой и социальной з...,Для работников добровольной пожарной охраны Ха...,simple,[{'file_path': '/home/cias/projects/other/raga...,True
118,68,Какие меры предпринимаются для восстановления ...,['| | | | | | | | | | | | |подсланевых вод и |...,"Для восстановления земель, подвергшихся загряз...",simple,[{'file_path': '/home/cias/projects/other/raga...,True


In [3]:
eval_data.groupby("evolution_type").count()

,Unnamed: 0,question,contexts,ground_truth,metadata,episode_done
evolution_type,,,,,,
conditional,30,30,30,30,30,30
multi_context,30,30,30,30,30,30
reasoning,30,30,30,30,30,30
simple,30,30,30,30,30,30


In [4]:
def get_system_responses(questions: list[str], ground_truth: list[str]):
    """
    Send requests to the /ask endpoint and collect responses.
    """
    responses = []
    for question, gt in tqdm.tqdm(zip(questions, ground_truth)):
        print("Question:", question)
        try:
            response = requests.post("http://localhost:8000/ask/", json={"text": question})
            response.raise_for_status()
            response_data = response.json()

            print("Response data:", response_data)
            responses.append(
                {
                    "question": question,
                    "response": response_data["llm_response"],
                    "ground_truth": gt,
                    "context": response_data["contexts"], 
                }
            )
        except Exception as e:
            print(f"Error while sending query: {question}, error message: {e}")
            continue
    return responses

In [5]:
# Test questoning
question = eval_data['question'][0]
gt = eval_data['ground_truth'][0]

get_system_responses([question], [gt])

1it [00:00, 128.70it/s]

Question: Зачем проводят конкурс среди оленеводов в ХМАО?
Error while sending query: Зачем проводят конкурс среди оленеводов в ХМАО?, error message: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /ask/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x16a724eb0>: Failed to establish a new connection: [Errno 61] Connection refused'))


[]

In [11]:
# пиздец
import logging
from contextlib import asynccontextmanager

import yaml
from fastapi import FastAPI


config_path = "../../interface/config.yaml"
with open(config_path, "r") as file:
    config = yaml.safe_load(file)


In [13]:
config['data_processing']['chunker']

{'py_class': 'interface.chunker.RecursiveCharacterTextSplitterChunker',
 'kwargs': {'chunk_size': 2048,
  'chunk_overlap': 128,
  'separators': ['\\n#{1,6} ',
   '```\\n',
   '\\n\\\\*\\\\*\\\\*+\\n',
   '\\n---+\\n',
   '\\n___+\\n',
   '\\n\\n',
   '\\n',
   ' ',
   '',
   ',',
   '|']}}

In [20]:
from pydoc import locate


chunker_cls = locate(config["data_processing"]["chunker"]["py_class"])
chunker = chunker_cls(**config["data_processing"]["chunker"]["kwargs"])

text_transformers = [
    c["py_class"](**c.get("kwargs"))
    for c in config["data_processing"].get("text_transformers", [])
]
chunk_transformers = [
    c["py_class"](**c.get("kwargs"))
    for c in config["data_processing"].get("chunk_transformers", [])
]

In [21]:
text_transformers, chunk_transformers

([], [])

In [23]:
eval_data.question[:1]

0    Зачем проводят конкурс среди оленеводов в ХМАО?
Name: question, dtype: object

In [10]:
system_responses = get_system_responses(test_data)

1it [00:04,  4.35s/it]

Error while sending query: Каков объем экспорта услуг категории "Поездки" в региональном проекте "Экспорт услуг" категории "Поездки" в Ханты-Мансийском автономном округе - Югре?, error message: string indices must be integers, not 'str'


2it [00:06,  2.88s/it]

Error while sending query: Каковы были основные мероприятия, реализуемые Правительством Ханты-Мансийского автономного округа - Югры в 2011 году?, error message: string indices must be integers, not 'str'


3it [00:08,  2.63s/it]

...

18it [00:38,  1.95s/it]

Error while sending query: Какова структура Департамента общественных и внешних связей Ханты-Мансийского автономного округа - Югры?, error message: string indices must be integers, not 'str'


19it [00:41,  2.13s/it]

Error while sending query: Какие меры социальной поддержки в сфере здравоохранения установлены законодательством Ханты-Мансийского автономного округа?, error message: string indices must be integers, not 'str'


19it [00:42,  2.25s/it]

KeyboardInterrupt

